<a href="https://colab.research.google.com/github/napatswift/wv-th-bdgt/blob/main/example/create_validation_spreadsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/napatswift/wv-th-bdgt

Cloning into 'wv-th-bdgt'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 118 (delta 39), reused 100 (delta 30), pack-reused 0
Receiving objects: 100% (118/118), 24.34 KiB | 3.48 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [2]:
%cd wv-th-bdgt
!pip install -r requirements.txt

/content/wv-th-bdgt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 49.3 MB/s eta 0:00:00


In [3]:
from model import BudgetItem
import pandas as pd

We can build the tree from dictionary (if it's in JSON file, read and `load` it using json library.) And then call `to_table_rows` method to generate speadsheet rows.

The method `to_table_row` will also compute the validation step and add error messsage to the column `error_message`.

In [5]:
rows = BudgetItem.from_json({
    'budget_type': 'MINISTRY',
    'name': 'ministry of magic',
    'amount': 100,
    'document': 'path/to/doc.pdf',
    'page': 1,
    'fiscal_year_budget': [],
    'children': [
        {
            'budget_type': 'BUDGETARY_UNIT',
            'name': 'this is a sample',
            'amount': 3240,
            'document': 'path/to/doc.pdf',
            'page': 1,
            'fiscal_year_budget': [{
                'year': 2556,
                'amount': 30000,
            }],
        }
    ],
}).to_rows()

After we've created the rows from the tree, we can now build the dataframe using pandas.

In [9]:
df = pd.DataFrame(rows)
name_cols = [col for col in df.columns if col.startswith('name_')]
name_cols.sort()
cols = ['error_message', 'budget_type', *name_cols, 'fiscal_year', 'fiscal_year_end', 'amount', 'document','page']
df = df[cols]

In [10]:
df

,error_message,budget_type,name_1,name_2,fiscal_year,fiscal_year_end,amount,document,page
0,While checking sum: amount of ministry of magi...,MINISTRY,ministry of magic,NaN,NaN,NaN,100,path/to/doc.pdf,1.0
1,,BUDGETARY_UNIT,NaN,this is a sample,NaN,NaN,3240,path/to/doc.pdf,1.0
2,,FISCAL_YEAR_BUDGET,NaN,2556,2556.0,2556.0,30000,NaN,NaN


We can save the dataframe to storage and validate later.